# DATA MUNGING


El objetivo de este script es, partiendo de los .csv generados por el parseador, llegar a nuevos .csv en los cuales la información esté ordenada tal y como precisan los distintos objetos generadores de figuras publicables o de resultados de análisis estadísticos. 

---
No voy a hacer nada al respecto de las muestras que aparecen marcadas como sospechosas: si no han sido descartadas y han pasado el filtrado, me las creo.

Lo primero es una comprobación de que los datos a emplear sean realmente los datos definitivos vía ver si alguno de los controles internos ha amplificado y, por ende, esas muestras han de descartarse

In [1]:
import pandas as pd
import numpy as np

In [2]:
filtered_data = pd.read_csv("../data/resistome_data/clean_data/filtered_arg") # los arg positivos
filtered_control = pd.read_csv("../data/resistome_data/clean_data/filtered_controls") #los controles positivos

# De entrada, hay mucha información que me sobra, pues para este gráfico sólo necesito las siguientes 4 variables
filtered_data = filtered_data[["Sample", "Assay", "Ct", "chip"]]
filtered_control = filtered_control[["Sample", "Assay", "Ct", "chip"]] 

# A continuación, voy a quitar aquellos valores con positivo en GENO vía left outer join excluyendo lo común
before = filtered_data.shape[0]
filtered_data = pd.merge(filtered_data, filtered_control, how = "outer", on = ["Assay", "chip"], indicator = True)
common = filtered_data.loc[filtered_data["_merge"] == "both"].shape[0]
filtered_data = filtered_data.loc[filtered_data["_merge"] == "left_only"]
after = filtered_data.shape[0]
print ("There were ", before, "amplifications. ", common, " were false positives. ", after, " remain.")
filtered_data.head()

del before
del common
del after

There were  2456 amplifications.  6  were false positives.  2450  remain.


Como se puede observar, el tamaño del df con todos los genes tras el JOIN es disinto, por lo que hemos descartado 6 amplificaciones por motivos de control interno.
A destacar que JOIN ha cambiado el nombre de las columnas, añadiendo la terminación "\_x" al df izquierdo e "\_y" al derecho, por lo que toca quitarlo en cuanto tenga sentido en pos de un a mayor claridad. 

---
Procedo juntar el df "resultante" con el comercial de la lista de genes. El método es muy similar, un join, solo que en este caso me interesa conservar lo compartido, asi que [full outer join](https://blog.codinghorror.com/a-visual-explanation-of-sql-joins/)

**NOTA**: el listado de genes proviene de una hoja formato .xlxs que previamente convertí a .csv y en la cual llevé a cabo la "limpieza" con LibreOffice Calc, pues la información estaba dispuesta visualmente, para que fuera fácil de entender por personas, no tanto pandas. No he cambiado nada, más que reorganizar columnas y borrar cosas como el logo 

In [3]:
arg_names = pd.read_csv("../data/resistome_data/metadada/a_names.csv", sep = ",")
arg_names.head()

,Marqueur,Origine,Antibiotique,Forward,Reverse,Plaque,Puits,Informations complémentaires
0,aacC2,deactivate,Aminoglycoside,ACGGCATTCTCGATTGCTTT,CCGAGCTTCACGTAAGCATTT,Pl1,A1,NaN
1,aacA/aphD,deactivate,Aminoglycoside,AGAGCCTTGGGAAGATGAAGTTT,TTGATCCATACCATAGACTATCTCATCA,Pl1,B1,NaN
2,aac(6')-II,deactivate,Aminoglycoside,CGACCCGACTCCGAACAA,GCACGAATCCTGCCTTCTCA,Pl1,C1,NaN
3,aphA3,deactivate,Aminoglycoside,AAAAGCCCGAAGAGGAACTTG,CATCTTTCACAAAGATGTTGCTGTCT,Pl1,D1,NaN
4,sat4,deactivate,Aminoglycoside,GAATGGGCAAAGCATAAAAACTTG,CCGATTTTGAAACCACAATTATGATA,Pl1,E1,NaN


In [4]:
# Nuevamente, del df para hacer el gráfico me interesa conservar la información justa y necesaria
arg_names = arg_names[["Marqueur", "Antibiotique"]]
arg_names.rename(columns = {"Marqueur": "Assay", "Antibiotique": "antib"}, inplace = True) # Una cosa es programar en spanglish y otra en francés
arg_names.tail()

,Assay,antib
377,dfrAB4,trimethoprim
378,dfrC,trimethoprim
379,dfrG,trimethoprim
380,dfrK,trimethoprim
381,dfrBmulti,trimethoprim


In [5]:
# Antes de hacer nada, reorganizar filtered_data quitando lo sobrante
filtered_data.drop(columns = ["Sample_y", "Ct_y", "chip", "_merge"], axis = 1, inplace = True) #chip sobra, basta con sample. _merge ya no nos es útil
filtered_data.rename(columns = {"Sample_x": "sample", "Ct_x": "Ct"}, inplace = True)
filtered_data.head()

,sample,Assay,Ct
0,7,aph4ib,26.05
1,8,aph4ib,26.36
2,9,aph4ib,25.33
3,7,spcN,26.89
4,9,spcN,26.21


In [6]:
# Y ya por fin, llevar a cabo el join
full_data = pd.merge(arg_names, filtered_data, how = "outer", on = ["Assay"])
full_data.tail()

,Assay,antib,sample,Ct
2481,"blaB-11,13,14",NaN,20.0,24.33
2482,"blaB-11,13,14",NaN,21.0,24.16
2483,"blaB-11,13,14",NaN,23.0,22.58
2484,pBS228-IncP-1?,NaN,20.0,21.58
2485,pBS228-IncP-1?,NaN,21.0,20.57


En ese .tail() se puede observar un hecho bastante preocupante: ¿qué hacen ahí dos amplificaciones de GENO? No las había pillado hasta ahora porque se encuentran en "Assay", en la columna que contiene los nombres de los ARG testeados, y la verdad el que estén ahí suscita preguntas sobre el diseño del chip que se me pueden estar escapando (recordemos que el cómo está diseñado el chip y cómo analizar sus resultados lo he sacado por ingeniería inversa, que los del servicio externo cobraban como un extra aparte el análisis y por ende esa información tan esencial no la comparten)

<mark>NOTA: están dentro de muestras marcadas como 000NTC. No entiendo nada, las ignoro</mark>

In [7]:
full_data = full_data[~full_data["Assay"].isin(["GENO"])] #también vuelven a escena los 16S por meter los Assays
full_data = full_data[~pd.isnull(full_data["antib"])]
full_data = full_data[(full_data["antib"] != "16S")]
print(full_data.shape[0])
print(full_data["antib"].unique())

2452
['Aminoglycoside' 'Amphenicol' 'Vancomycin' 'Beta Lactam' 'beta Lactam'
 'Beta lactamase' 'MDR' 'MDR-chromo' 'other' 'Fluoroquinolone'
 'fluoroquinolone' 'MLSB' 'Phenicol' 'Tetracycline' 'Tetracycline '
 'Insertional' 'Insertional ' 'Integrase' 'MDR-mobile' 'MGE' 'plasmid'
 'Plasmid-inc' 'Plasmid-inc ' 'Plasmid-rep' 'Sulfonamide' 'Transposase'
 'trimethoprim']


Como se puede ver en la lista de "antibióticos" (que en realidad también incluye MGE), hay muchos nombres mal puestos, así como valores de 16S que habría que quitar antes de seguir. Por tanto, hay que limpiar antes de seguir adelante

In [8]:
full_data.loc[(full_data["antib"] == "MDR-chromo") | (full_data["antib"] == "MDR-mobile"), "antib"] = "MDR"
full_data.loc[(full_data["antib"] == "Beta lactamase") | (full_data["antib"] == "beta Lactam"), "antib"] = "Beta Lactam"
full_data.loc[(full_data["antib"] == "trimethoprim"), "antib"] = "Trimethoprim"
full_data.loc[(full_data["antib"] == "Tetracycline "), "antib"] = "Tetracycline"
full_data.loc[(full_data["antib"] == "fluoroquinolone"), "antib"] = "Fluoroquinolone"
full_data.loc[(full_data["antib"] == "other"), "antib"] = "Other"
full_data.loc[(full_data["antib"] == "Insertional "), "antib"] = "Insertional"
full_data.loc[(full_data["antib"] == "Plasmid-inc "), "antib"] = "Plasmid-inc"
full_data.loc[(full_data["antib"] == "plasmid"), "antib"] = "Plasmid"

print(full_data["antib"].unique())

['Aminoglycoside' 'Amphenicol' 'Vancomycin' 'Beta Lactam' 'MDR' 'Other'
 'Fluoroquinolone' 'MLSB' 'Phenicol' 'Tetracycline' 'Insertional'
 'Integrase' 'MGE' 'Plasmid' 'Plasmid-inc' 'Plasmid-rep' 'Sulfonamide'
 'Transposase' 'Trimethoprim']


Un último apaño que puedo hacer en este punto es desglosar la información contenida en "sample": yo sé que el sample 1 se corresponde con la reṕlica 1 de **suelo** del **uruguay**, por ejemplo. Asi que podría crear columnas que indicaran el tipo general de muestra (plástico/control), su ubicación (uruguay/ionosférico) e incluso su tipo específico (EPS/PUR/Agua/Suelo), para así poder agrupar mucho más fácilmente de cara a futuros análisis/representaciones. Para ello, es útil tener esto a mano:

chip | sample | Desc.
---:|:---:|---
**1** | 1 | URU Suelo 1
// | 2 | URU Suelo 2
// | 3 | URU Suelo 3
**2** | 4 | URU Agua 1
// | 5 | URU Agua 2
// | 6 | URU Agua 3
**3** | 7 | URU EPS 1
// | 8 | URU EPS 2
// | 9 | URU EPS 3
**4** | 10 | URU PUR 1
// | 11 | URU PUR 2
// | 12 | URU PUR 3
**5** | 13 | IÓN Suelos 1
// | 14 | IÓN Agua 1
// | 15 | IÓN EPS 1
**6** | 16 | IÓN EPS 2
// | 17 | IÓN PUR 1
// | 18 | IÓN PUR 2
**7** | 19 | ARDLEY Suelos 1
// | 20 | ARDLEY Suelos 2
// | 21 | ARDLEY Suelos 3
**8** | 22 | ARDLEY EPS 1
// | 23 | ARDLEY EPS 2
// | 24 | ARDLEY EPS 3
**9** | 25 | ARDLEY PUR 1
// | 26 | ARDLEY PUR 2
// | 27 | ARDLEY PUR 3

Hay otra clasificación más que podemos hacer aquí: entre ARG y MGE, pues ahora todo está en la columna "antib" y nos interesará separar en un futuro. Aquí la lista de ARG / MGE:
- **ARG**: Aminoglycoside, Amphenicol, Vancomycin, Beta Lactam, MDR, Fluoroquinolone, MLSB, Phenicol, Tetracycline, Sulfonamide, Trimethoprim
- **MGE**: MGE, Insertional, Plasmid, Plasmid-inc, Plasmid-rep, Transposase, Other, Integrase

In [9]:
#primero, definir puntos de muestreo
full_data.loc[(full_data["sample"] < 13), "place"] = "uru"
full_data.loc[(full_data["sample"] > 12) & (full_data["sample"] < 19), "place"] = "ion"
full_data.loc[(full_data["sample"] > 18), "place"] = "ardley"

#segundo, definir tipo fino
full_data.loc[(full_data["sample"].isin([1, 2, 3, 13, 19, 20, 21])), "type_f"] = "soil"
full_data.loc[(full_data["sample"].isin([4, 5, 6, 14])), "type_f"] = "water"
full_data.loc[(full_data["sample"].isin([7, 8, 9, 15, 16, 22, 23, 24])), "type_f"] = "EPS"
full_data.loc[(full_data["sample"].isin([10, 11, 12, 17, 18, 25, 26, 27])), "type_f"] = "PUR"

#tercero, tipo general
full_data.loc[(full_data["type_f"] == "EPS") | (full_data["type_f"] == "PUR"), "type_g"] = "plastic"
full_data.loc[(full_data["type_f"] == "water") | (full_data["type_f"] == "soil"), "type_g"] = "control"

#Y por último, ARG vs MGE
arg_lst = ["Aminoglycoside", "Amphenicol", "Vancomycin", "Beta Lactam", "MDR", 
           "Fluoroquinolone", "MLSB", "Phenicol", "Tetracycline", "Sulfonamide", 
           "Trimethoprim"]
full_data.loc[(full_data["antib"].isin(arg_lst)), "or_seq"] = "arg"
full_data.loc[(~full_data["antib"].isin(arg_lst)), "or_seq"] = "mge"

#y chequeo rápido
check_data = full_data.dropna()
print(check_data.tail())
del check_data


          Assay         antib  sample     Ct   place type_f   type_g or_seq
2449  dfrBmulti  Trimethoprim    26.0  15.72  ardley    PUR  plastic    arg
2450  dfrBmulti  Trimethoprim    27.0  15.77  ardley    PUR  plastic    arg
2451  dfrBmulti  Trimethoprim    22.0  16.84  ardley    EPS  plastic    arg
2452  dfrBmulti  Trimethoprim    23.0  17.13  ardley    EPS  plastic    arg
2453  dfrBmulti  Trimethoprim    24.0  17.15  ardley    EPS  plastic    arg


/tmp/ipykernel_9382/2864138303.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'uru' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  full_data.loc[(full_data["sample"] < 13), "place"] = "uru"
/tmp/ipykernel_9382/2864138303.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'soil' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  full_data.loc[(full_data["sample"].isin([1, 2, 3, 13, 19, 20, 21])), "type_f"] = "soil"
/tmp/ipykernel_9382/2864138303.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'plastic' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  full_data.loc[(full_data["type_f"] == "EPS") | (full_data["type_f"] == "PUR"), "type_g"] = "pl

Aquí hay una primera divergencia en la biliografía.
- Por un lado, hay trabajos como [Jiang et al, 2023](https://www.sciencedirect.com/science/article/pii/S0048969723007556) y [Do et al, 2023](https://www.biorxiv.org/content/10.1101/2022.02.25.481976v1.abstract) (ojo que este es una preprint) que proceden a calcular el nº de copia relativo (desarrollo abajo) y con él, lo dividen entre el nº de copia del 16S para normalizarlo y analizan los resultados en base a esa nueva variable. 
- Por otro lado, hay trabajos PON EJEMPLOS en los que para comparar la expresión relativa entre condiciones de estudio se calcula el fold-change vía delta delta Ct.
- \* hay incluso papers en los que calculan las dos cosas y luego no especifican cuál emplean

---
De primeras voy con el primer método, el de [Jiang et al, 2023](https://www.sciencedirect.com/science/article/pii/S0048969723007556).

En él, se hace uso de la [fórmula de Looft et al, 2012](https://www.pnas.org/doi/abs/10.1073/pnas.1120238109) para obtener _RN_, el nº de copias relativo:

$$
RN = 10^{\frac{27-Ct}{10/3}}
$$
Tomamos 27 como límite de detección (Looft y Jiang emplearon distintos límites y por ende su fórmula es distinta)


RN se obtiene tanto para cada ARGvMGE y para el gen de housekeeping (en nuestro caso, el 16S), para finalmente normalizar el uno con el otro. En nuestro caso:

$$
RA = \frac{RN_{ARG}}{RN_{16S}}
$$

Siendo _RA_ la abundancia relativa (_Relative Abundance_), la variable que emplearemos en los análisis estadísticos de ahora en adelante. Este procedimiento es de Looft entero, ojo

In [10]:
full_data["rel_n"] = (10 ** ((27 - full_data["Ct"])/(10/3))) #esto añade el valor
#full_data.head()
check_df = full_data["Ct"].dropna() # chequeo de esta manera pq si no sólo vería valores de Ct = 0
full_data["rel_n"].fillna(0, inplace = True) #para poder seguir operando
check_df.head()

0    23.78
1    22.51
2    25.91
3    24.56
4    26.05
Name: Ct, dtype: float64

In [11]:
del check_df #ya ha cumplido su función

Una vez se tiene el nº de copia relativo, lo siguiente es obtener el "absoluto" dividiendo el nº de copia relativo de cada gen entre el nº de copia relativo del 16S. O lo que es lo mismo, toca abrir los datos del 16S.

Como viene siendo el caso, no bastará con abrirlos y punto, sino que tendré que quedarme sólo con la información de interés (quitando columnas de más y los 000NTC y GENO), así como ver qué hacer en aquellos casos en los que uno de los valores no sirva. Lo suyo sería usar siempre el mismo gen de 16S de los dos, o al menos el mismo para cada muestra

In [12]:
arnr_data = pd.read_csv("../data/resistome_data/clean_data/16S_refs")
arnr_data = arnr_data[["Sample", "Assay", "Ct", "chip", "Efficiency", "N"]] #dejo la eficiencia y N para poder chequear qué valores son los apropiados
arnr_data = arnr_data.loc[~arnr_data["Sample"].isin(["000NTC", "GENO"])] #quito esto
arnr_data.dropna(inplace = True)
print(arnr_data["Sample"].unique()) #para chequear que haya amplificación en todas las muestras
arnr_data.sort_values(by = "Sample")

['7' '8' '9' '25' '26' '27' '13' '14' '15' '10' '11' '12' '4' '5' '6' '1'
 '2' '3' '19' '20' '22' '23' '24' '16' '17' '18']


,Sample,Assay,Ct,chip,Efficiency,N
55,1,16S new 2,12.22,1,2.24,3
50,1,16S old 1,13.58,1,1.68,2
30,10,16S old 1,13.18,4,1.87,3
31,11,16S old 1,13.22,4,1.88,3
32,12,16S old 1,13.73,4,1.90,3
37,12,16S new 2,12.39,4,2.08,3
20,13,16S old 1,14.40,5,1.92,3
21,14,16S old 1,15.42,5,1.89,3
22,15,16S old 1,13.29,5,1.88,3
80,16,16S old 1,13.64,6,1.84,3


En base a lo que se observa en el df, aunque hay amplificación en todas las muestras, fallan dos detalles:
- No hay suficiente 16S new para usarlo con todas las muestras, pero sí 16S old 1
- Si filtro por eficiencia como hago de normal, pierdo la muestra 1 y 3, pero por muy, muy poco tanto en new como en old
Por tanto, por ahora mantengo el 16S old 1, aunque "falle" en dos muestras, y uso esos valores 

In [13]:
arnr_data = arnr_data.loc[~arnr_data["Assay"].isin(["16S new 2"])] #dejar sólo 16S old 1
#print(arnr_data["Sample"].unique()) #para chequear que siga siendo útil

arnr_data["rel_n"] = (10 ** ((27 - arnr_data["Ct"])/(10/3))) #para obtener el nº de copia relativo del 16S
arnr_data = arnr_data[["Sample", "rel_n"]] # sólo lo provechoso
arnr_data["Sample"] = pd.to_numeric(arnr_data["Sample"], errors = "coerce")
arnr_data.set_index("Sample", inplace = True) # necesario por cómo funciona la función .to_dict()
arnr_vals = arnr_data.to_dict()["rel_n"] #aquí el diccionario
del arnr_data #ya ha cumplido su función
arnr_vals

{7: 8933.054837332944,
 8: 10990.05839432522,
 9: 7726.805850957021,
 25: 63533.09318517423,
 26: 59703.52865838366,
 27: 47206.304126359,
 13: 6025.595860743575,
 14: 2978.516429429188,
 15: 12971.792709839572,
 10: 13995.873225726178,
 11: 13614.446824659482,
 12: 9571.94071294844,
 4: 2558.585886905643,
 5: 4149.5404263436285,
 6: 1644.371723214929,
 1: 10616.955571987242,
 2: 18450.154191794736,
 3: 13335.21432163324,
 19: 45603.691595129596,
 20: 30549.21113215509,
 22: 25882.12915153093,
 23: 23496.32820848305,
 24: 21478.304741305332,
 16: 10185.91388054117,
 17: 13427.649611378642,
 18: 13708.817661648523}

Y ya obtener el valor de abundancia relativa (RA) _per se_

In [14]:
#si la key (nº muestra) es igual al de un valor dado, asigna el 16S correspondiente
for sample_num,rel_16 in arnr_vals.items():
    full_data.loc[(full_data["sample"] == sample_num), "rel_16"] = rel_16 
full_data["rel_ab"] = full_data["rel_n"]/full_data["rel_16"] 

check_df = full_data.dropna() #nuevamente, para chequear y poder ver algo que no sea NaN o 0
print(check_df.tail())
del check_df

          Assay         antib  sample     Ct   place type_f   type_g or_seq  \
2449  dfrBmulti  Trimethoprim    26.0  15.72  ardley    PUR  plastic    arg   
2450  dfrBmulti  Trimethoprim    27.0  15.77  ardley    PUR  plastic    arg   
2451  dfrBmulti  Trimethoprim    22.0  16.84  ardley    EPS  plastic    arg   
2452  dfrBmulti  Trimethoprim    23.0  17.13  ardley    EPS  plastic    arg   
2453  dfrBmulti  Trimethoprim    24.0  17.15  ardley    EPS  plastic    arg   

            rel_n        rel_16    rel_ab  
2449  2421.029047  59703.528658  0.040551  
2450  2338.837239  47206.304126  0.049545  
2451  1116.863248  25882.129152  0.043152  
2452   914.113241  23496.328208  0.038905  
2453   901.571138  21478.304741  0.041976  


Un problema que ha surgido al pasar de Ct a la abundancia relativa es que los valores de abundancia son ínfimos: apenas hay. No sólo eso, sino que también se da el caso que las diferencias entre ellos son muy grandes (al fin y al cabo, las diferencias de unidades de Cts se transforman en distancias exponenciales al pasar a nº relativo de copia). Para solucionarlo, empleo una transformación logarítmica

In [15]:
## Voy a calcular el log10 de manera chapucera, pues al calcularlo a la vez en todas las casillas de la columna
## y tener la mayoría de casillas rel_ab = NaN, lo que ocurrirá en la mayoría de casos es que el programa detectará
## automáticamente que le estoy pidiendo una imposibilidad matemática y simplemente se saltará ese cálculo.
## A efectos prácticos, el resultado final será el mismo, pues debiera valer 0 y le haré valer 0
full_data["log_n"] = np.log10(full_data["rel_ab"])
full_data.loc[(full_data["rel_ab"] == 0.0), "log_n"] = 0


#como de costumbre ya
check_df = full_data.dropna() #nuevamente, para chequear y poder ver algo que no sea NaN o 0
print(check_df.head())
del check_df

       Assay           antib  sample     Ct   place type_f   type_g or_seq  \
0      aacC2  Aminoglycoside    25.0  23.78  ardley    PUR  plastic    arg   
1      aacC2  Aminoglycoside    27.0  22.51  ardley    PUR  plastic    arg   
2      aacC2  Aminoglycoside    20.0  25.91  ardley   soil  control    arg   
3      aacC2  Aminoglycoside    22.0  24.56  ardley    EPS  plastic    arg   
4  aacA/aphD  Aminoglycoside    25.0  26.05  ardley    PUR  plastic    arg   

       rel_n        rel_16    rel_ab  log_n  
0   9.246982  63533.093185  0.000146 -3.837  
1  22.233099  47206.304126  0.000471 -3.327  
2   2.123244  30549.211132  0.000070 -4.158  
3   5.395106  25882.129152  0.000208 -3.681  
4   1.927525  63533.093185  0.000030 -4.518  


Llegados a este punto, ya tengo un df útil para hacer gráficos. En particular, para hacer diagramas de venn de 
presencia/ausencia de ARG/MGE,  diagramas de barras (o similares) de abundancia relativa y cálculos estadísticos.
Por tanto, guardo el df como .csv con el nombre **"ab_data_simple.csv"** (ab de abundancia relativa, data de data y simple porque es la versión mínima con toda la información posible)

In [16]:
#full_data.fillna(0, inplace = True)
full_data.to_csv("../data/resistome_data/clean_data/ab_data_simple.csv")

---
Hasta este momento he estado trabajando con datos "mínimos", véase, con un dataset en el que estuviera representada la información mínima necesaria para entender el global. Sin embargo, para según qué representaciones (de entrada, el heatmap) necesitaré tener en cuenta TODA la información generada: un df en el que se recoja la amplificación/falta de de todos los ARG y MGE **de todas las muestras**. 

Dado que tengo 18 muestras (este nº cambiará cuando reciba todos los resultados) y en torno a 429 arg/mge, debiera tener un df final de 6876 filas. 

In [17]:
big_full_data = pd.DataFrame() # el df final
amp_data = full_data.dropna()

for sample_num in amp_data["sample"].unique():
    # Extraer solo la info de los arg de cada muestra concreta
    sample_data = amp_data[(amp_data["sample"] == sample_num)]
    
    # Juntar ambos. Ojo, las celdas no compartidas no se rellenan
    sample_data = pd.merge(arg_names, sample_data, how = "outer", on = ["Assay"])
    
    # Sustituir los "nan" de las celdas vacías de sample por el sample_num en cuestión, para poder diferenciarlas
    sample_data.fillna({"sample": sample_num}, inplace = True)
    sample_data.fillna({"Ct": 0}, inplace = True) #ya que estoy también relleno el Ct
    
    # Unir el df provisional al denifinitivo y vuelta a empezar
    big_full_data = pd.concat([big_full_data, sample_data])

print(big_full_data.shape[0])
check_df = big_full_data.dropna()
print(check_df.tail())
del check_df

9932
                  Assay          antib_x          antib_y  sample     Ct  \
194            qepA_1_2  Fluoroquinolone  Fluoroquinolone    14.0  27.00   
283               tetPA    Tetracycline      Tetracycline    14.0  26.41   
299  intI1F165_clinical        Integrase        Integrase    14.0  25.19   
308                czcA       MDR-mobile              MDR    14.0  21.77   
340               trb-C          plasmid          Plasmid    14.0  26.80   

    place type_f   type_g or_seq      rel_n       rel_16    rel_ab  log_n  
194   ion  water  control    arg   1.000000  2978.516429  0.000336 -3.474  
283   ion  water  control    arg   1.503142  2978.516429  0.000505 -3.297  
299   ion  water  control    mge   3.491403  2978.516429  0.001172 -2.931  
308   ion  water  control    arg  37.068072  2978.516429  0.012445 -1.905  
340   ion  water  control    mge   1.148154  2978.516429  0.000385 -3.414  


Hay, de nuevo, una columna extra que podemos eliminar para ahorrar memoria, y otra que ha retenido un nombre inadecuado

In [18]:
big_full_data.drop(columns = ["antib_y"], axis = 1, inplace = True)
big_full_data.rename(columns = {"antib_x": "antib"}, inplace = True)

big_full_data

,Assay,antib,sample,Ct,place,type_f,type_g,or_seq,rel_n,rel_16,rel_ab,log_n
0,aacC2,Aminoglycoside,25.0,23.78,ardley,PUR,plastic,arg,9.246982,63533.093185,0.000146,-3.837
1,aacA/aphD,Aminoglycoside,25.0,26.05,ardley,PUR,plastic,arg,1.927525,63533.093185,0.000030,-4.518
2,aac(6')-II,Aminoglycoside,25.0,18.80,ardley,PUR,plastic,arg,288.403150,63533.093185,0.004539,-2.343
3,aphA3,Aminoglycoside,25.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sat4,Aminoglycoside,25.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
377,dfrAB4,trimethoprim,14.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378,dfrC,trimethoprim,14.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
379,dfrG,trimethoprim,14.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
380,dfrK,trimethoprim,14.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Y por cómo está programado, se hace necesario volver a asignar los valores de clasificación antes puestos (pues aunque no haya valores de Ct, sí que interesa saber qué tipo de muestras son)

In [19]:
#primero, definir puntos de muestreo
big_full_data.loc[(big_full_data["sample"] < 13), "place"] = "uru"
big_full_data.loc[(big_full_data["sample"] > 12) & (big_full_data["sample"] < 19), "place"] = "ion"
big_full_data.loc[(big_full_data["sample"] > 18), "place"] = "ardley"

#segundo, definir tipo fino
big_full_data.loc[(big_full_data["sample"].isin([1, 2, 3, 13, 19, 20, 21])), "type_f"] = "soil"
big_full_data.loc[(big_full_data["sample"].isin([4, 5, 6, 14])), "type_f"] = "water"
big_full_data.loc[(big_full_data["sample"].isin([7, 8, 9, 15, 16, 22, 23, 24])), "type_f"] = "EPS"
big_full_data.loc[(big_full_data["sample"].isin([10, 11, 12, 17, 18, 25, 26, 27])), "type_f"] = "PUR"

#tercero, tipo general
big_full_data.loc[(big_full_data["type_f"] == "EPS") | (big_full_data["type_f"] == "PUR"), "type_g"] = "plastic"
big_full_data.loc[(big_full_data["type_f"] == "water") | (big_full_data["type_f"] == "soil"), "type_g"] = "control"

#cuarto, corregir OTRA VEZ los nombres "defectuosos"
big_full_data.loc[(big_full_data["antib"] == "MDR-chromo") | (big_full_data["antib"] == "MDR-mobile"), "antib"] = "MDR"
big_full_data.loc[(big_full_data["antib"] == "Beta lactamase") | (big_full_data["antib"] == "beta Lactam"), "antib"] = "Beta Lactam"
big_full_data.loc[(big_full_data["antib"] == "trimethoprim"), "antib"] = "Trimethoprim"
big_full_data.loc[(big_full_data["antib"] == "Tetracycline "), "antib"] = "Tetracycline"
big_full_data.loc[(big_full_data["antib"] == "fluoroquinolone"), "antib"] = "Fluoroquinolone"
big_full_data.loc[(big_full_data["antib"] == "other"), "antib"] = "Other"
big_full_data.loc[(big_full_data["antib"] == "Insertional "), "antib"] = "Insertional"
big_full_data.loc[(big_full_data["antib"] == "Plasmid-inc "), "antib"] = "Plasmid-inc"
big_full_data.loc[(big_full_data["antib"] == "plasmid"), "antib"] = "Plasmid"

#y ya el tipo de secuencia
big_full_data.loc[(big_full_data["antib"].isin(arg_lst)), "or_seq"] = "arg"
big_full_data.loc[(~big_full_data["antib"].isin(arg_lst)), "or_seq"] = "mge"

big_full_data

,Assay,antib,sample,Ct,place,type_f,type_g,or_seq,rel_n,rel_16,rel_ab,log_n
0,aacC2,Aminoglycoside,25.0,23.78,ardley,PUR,plastic,arg,9.246982,63533.093185,0.000146,-3.837
1,aacA/aphD,Aminoglycoside,25.0,26.05,ardley,PUR,plastic,arg,1.927525,63533.093185,0.000030,-4.518
2,aac(6')-II,Aminoglycoside,25.0,18.80,ardley,PUR,plastic,arg,288.403150,63533.093185,0.004539,-2.343
3,aphA3,Aminoglycoside,25.0,0.00,ardley,PUR,plastic,arg,NaN,NaN,NaN,NaN
4,sat4,Aminoglycoside,25.0,0.00,ardley,PUR,plastic,arg,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
377,dfrAB4,Trimethoprim,14.0,0.00,ion,water,control,arg,NaN,NaN,NaN,NaN
378,dfrC,Trimethoprim,14.0,0.00,ion,water,control,arg,NaN,NaN,NaN,NaN
379,dfrG,Trimethoprim,14.0,0.00,ion,water,control,arg,NaN,NaN,NaN,NaN
380,dfrK,Trimethoprim,14.0,0.00,ion,water,control,arg,NaN,NaN,NaN,NaN


In [20]:
#ya se puede pasar a csv
big_full_data.to_csv("../data/resistome_data/clean_data/ab_data_all.csv")

---
Necesito salir de un apuro mientras esperamos a que desde Francia nos manden unas cosas. Asi que voy a crearme un dataset falseado

In [21]:
arnr_vals[21] = (arnr_vals[19] + arnr_vals[20])/2
amp_data = full_data[(~full_data["antib"].isna())]
amp_data = amp_data[(~amp_data["sample"].isna())]

for sample_num,rel_16 in arnr_vals.items():
    amp_data.loc[(amp_data["sample"] == sample_num), "rel_16"] = rel_16 
amp_data["rel_ab"] = amp_data["rel_n"]/amp_data["rel_16"] 


amp_data["log_n"] = np.log10(amp_data["rel_ab"])
amp_data.loc[(amp_data["rel_ab"] == 0.0), "log_n"] = 0

In [22]:
big_false_data = pd.DataFrame() # el df final

for sample_num in amp_data["sample"].unique():
    sample_data = amp_data[(amp_data["sample"] == sample_num)]
    sample_data = pd.merge(arg_names, sample_data, how = "outer", on = ["Assay"])
    sample_data.fillna({"sample": sample_num}, inplace = True)
    sample_data.fillna({"Ct": 0}, inplace = True)
    big_false_data = pd.concat([big_false_data, sample_data])

big_false_data.drop(columns = ["antib_y"], axis = 1, inplace = True)
big_false_data.rename(columns = {"antib_x": "antib"}, inplace = True)

#primero, definir puntos de muestreo
big_false_data.loc[(big_false_data["sample"] < 13), "place"] = "uru"
big_false_data.loc[(big_false_data["sample"] > 12) & (big_false_data["sample"] < 19), "place"] = "ion"
big_false_data.loc[(big_false_data["sample"] > 18), "place"] = "ardley"

#segundo, definir tipo fino
big_false_data.loc[(big_false_data["sample"].isin([1, 2, 3, 13, 19, 20, 21])), "type_f"] = "soil"
big_false_data.loc[(big_false_data["sample"].isin([4, 5, 6, 14])), "type_f"] = "water"
big_false_data.loc[(big_false_data["sample"].isin([7, 8, 9, 15, 16, 22, 23, 24])), "type_f"] = "EPS"
big_false_data.loc[(big_false_data["sample"].isin([10, 11, 12, 17, 18, 25, 26, 27])), "type_f"] = "PUR"

#tercero, tipo general
big_false_data.loc[(big_false_data["type_f"] == "EPS") | (big_false_data["type_f"] == "PUR"), "type_g"] = "plastic"
big_false_data.loc[(big_false_data["type_f"] == "water") | (big_false_data["type_f"] == "soil"), "type_g"] = "control"

#cuarto, corregir OTRA VEZ los nombres "defectuosos"
big_false_data.loc[(big_false_data["antib"] == "MDR-chromo") | (big_false_data["antib"] == "MDR-mobile"), "antib"] = "MDR"
big_false_data.loc[(big_false_data["antib"] == "Beta lactamase") | (big_false_data["antib"] == "beta Lactam"), "antib"] = "Beta Lactam"
big_false_data.loc[(big_false_data["antib"] == "trimethoprim"), "antib"] = "Trimethoprim"
big_false_data.loc[(big_false_data["antib"] == "Tetracycline "), "antib"] = "Tetracycline"
big_false_data.loc[(big_false_data["antib"] == "fluoroquinolone"), "antib"] = "Fluoroquinolone"
big_false_data.loc[(big_false_data["antib"] == "other"), "antib"] = "Other"
big_false_data.loc[(big_false_data["antib"] == "Insertional "), "antib"] = "Insertional"
big_false_data.loc[(big_false_data["antib"] == "Plasmid-inc "), "antib"] = "Plasmid-inc"
big_false_data.loc[(big_false_data["antib"] == "plasmid"), "antib"] = "Plasmid"

#y ya el tipo de secuencia
big_false_data.loc[(big_false_data["antib"].isin(arg_lst)), "or_seq"] = "arg"
big_false_data.loc[(~big_false_data["antib"].isin(arg_lst)), "or_seq"] = "mge"

big_false_data["sample"].unique()
big_false_data.to_csv("../data/resistome_data/clean_data/ab_data_false.csv")
big_false_data

,Assay,antib,sample,Ct,place,type_f,type_g,or_seq,rel_n,rel_16,rel_ab,log_n
0,aacC2,Aminoglycoside,25.0,23.78,ardley,PUR,plastic,arg,9.246982,63533.093185,0.000146,-3.837
1,aacA/aphD,Aminoglycoside,25.0,26.05,ardley,PUR,plastic,arg,1.927525,63533.093185,0.000030,-4.518
2,aac(6')-II,Aminoglycoside,25.0,18.80,ardley,PUR,plastic,arg,288.403150,63533.093185,0.004539,-2.343
3,aphA3,Aminoglycoside,25.0,0.00,ardley,PUR,plastic,arg,NaN,NaN,NaN,NaN
4,sat4,Aminoglycoside,25.0,0.00,ardley,PUR,plastic,arg,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
377,dfrAB4,Trimethoprim,14.0,0.00,ion,water,control,arg,NaN,NaN,NaN,NaN
378,dfrC,Trimethoprim,14.0,0.00,ion,water,control,arg,NaN,NaN,NaN,NaN
379,dfrG,Trimethoprim,14.0,0.00,ion,water,control,arg,NaN,NaN,NaN,NaN
380,dfrK,Trimethoprim,14.0,0.00,ion,water,control,arg,NaN,NaN,NaN,NaN
